In [1]:
import pandas as pd
import numpy as np
import aggregation
import cleaning

In [3]:
# open elections_with_tracts.xlsx
elections_with_tracts = pd.read_excel('elections_with_tracts.xlsx')
# open acs_data.csv
acs_tract_data = pd.read_csv('tract_acs_data.csv')
# open statistics.xlsx
statistics = pd.read_excel('statistics.xlsx')

C:\Users\User\AppData\Local\Temp\ipykernel_18952\3175199637.py:4: DtypeWarning: Columns (7,20,25,26,27,54,60,70,90,93,102,127,131,142,144,147,150,157,166,176,178,192,214,217,221,226,230,255,257,259,276,279,293,296,302,312,344,355,356,366,418,430,433,436,444,455,463,466,469,473,481,490,499,500,503,510,523,539,554,567,574,579,586,615,622,647,651,652,661,682,698,700,701,731,743,756,761) have mixed types. Specify dtype option on import or set low_memory=False.
  acs_tract_data = pd.read_csv('tract_acs_data.csv')


In [4]:
# replace all (X), -, or N with nan
cleaning.replace_error_values(acs_tract_data)
cleaning.combine_percent_columns(acs_tract_data)
cleaning.remove_invalid_columns(acs_tract_data)
cleaning.remove_nonaggregatables(acs_tract_data)
acs_tract_data

,geocode,year,DP05_0026E,DP02_0047E,DP03_0107E,DP03_0109E,DP03_0097E,DP03_0016E,DP02_0099E,DP03_0061E,...,DP05_0012E,DP02_0072E,DP02_0138E,DP02_0112E,DP02_0065E,DP02_0055E,DP02_0129E,DP03_0078E,DP02_0069E,DP03_0087E
0,06_001_422200,2010,163.0,0.0,36.0,162.0,2404.0,252.0,16.0,118.0,...,156.0,508.0,65.0,861.0,709.0,170.0,91.0,10.0,87.0,108435.0
1,06_001_422400,2010,156.0,12.0,0.0,52.0,4208.0,154.0,67.0,76.0,...,235.0,192.0,22.0,1734.0,1104.0,75.0,78.0,16.0,58.0,82837.0
2,06_001_422300,2010,198.0,0.0,102.0,96.0,2962.0,320.0,70.0,84.0,...,149.0,608.0,58.0,560.0,855.0,218.0,84.0,0.0,161.0,119495.0
3,06_001_420400,2010,0.0,0.0,88.0,36.0,2972.0,379.0,65.0,25.0,...,52.0,1180.0,72.0,1997.0,1404.0,390.0,235.0,98.0,25.0,52104.0
4,06_001_421800,2010,166.0,0.0,63.0,69.0,1792.0,312.0,21.0,158.0,...,170.0,452.0,67.0,268.0,728.0,177.0,53.0,0.0,91.0,143058.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6497,02_170_000101,2022,286.0,0.0,18.0,22.0,250.0,29.0,0.0,24.0,...,55.0,57.0,0.0,505.0,60.0,0.0,6.0,6.0,470.0,93004.0
6498,02_050_000200,2022,2428.0,0.0,402.0,255.0,3277.0,1024.0,7.0,277.0,...,630.0,466.0,0.0,5765.0,365.0,217.0,27.0,40.0,4403.0,127876.0
6499,50_007_003600,2023,1824.0,0.0,546.0,107.0,3649.0,318.0,51.0,102.0,...,483.0,715.0,53.0,4834.0,1346.0,111.0,0.0,97.0,4448.0,92189.0
6500,50_007_000800,2023,889.0,0.0,81.0,18.0,2051.0,341.0,0.0,239.0,...,286.0,200.0,16.0,2348.0,729.0,19.0,30.0,8.0,1962.0,230038.0


In [5]:
# open elections.xlsx
elections = pd.read_excel('elections.xlsx')

In [15]:
aggregated_data = []

# create df called missing_data with columns filename, year, values, num_missing, num_total, missing_tracts
missing_data = pd.DataFrame(columns=['filename', 'year', 'variable', 'values', 'num_missing', 'num_total', 'prop_missing'])

for index, row in elections_with_tracts.iterrows():
    if index % 10 == 0:
        print(f'{index} of {len(elections_with_tracts)} rows processed')

    tracts = row['tracts'].split(',')
    year = elections[elections['filename'] == row['filename']]['year'].values[0]
    
    # get rows where geocode is in tracts and year = year and remove 0 values
    percent_dict = {tract: percent for tract, percent in zip(tracts, list(map(float, row['percents'].split(','))))}
    tract_rows = acs_tract_data[(acs_tract_data['geocode'].isin(tracts)) & (acs_tract_data['year'] == year) & acs_tract_data['DP05_0001E'] > 0]
    percents = pd.DataFrame({'percent': [percent_dict[geocode] for geocode in tract_rows['geocode']]})
    tract_rows = pd.concat([tract_rows.reset_index(drop=True), percents], axis=1)
    tract_rows = tract_rows[tract_rows['percent'] > 0]

    election_data = {}

    for column in acs_tract_data.columns[2:]:
        # get the values in tract_rows for column
        values = tract_rows[column].values
        valid_values = values[~np.isnan(values)]

        # if all values are NaN, continue
        if all(np.isnan(values)):
            election_data[column] = np.nan
        else:
            # if at least one but not all values are NaN, add to missing data
            if any(np.isnan(values)):
                missing_data.loc[len(missing_data)] = [row['filename'], year, column, values, sum(np.isnan(values)), len(values), sum(np.isnan(values)) / len(values)]

            if column in statistics['agg_var'].values:
                pop_var = statistics[statistics['agg_var'] == column]['pop_var'].values[0]
                # if pop_var is none skip
                if pd.isnull(pop_var):
                    continue

                # if pop_var contains -
                if '-' in pop_var:
                    # split on -
                    pop_vars = pop_var.split('-')
                    pops = tract_rows[pop_vars[0]].values - tract_rows[pop_vars[1]].values
                else:
                    pops = tract_rows[pop_var].values
                
                percents = tract_rows['percent'].values
                tract_pops = pops*percents
                valid_tract_pops = tract_pops[~np.isnan(values)]

                # if sum valid_tract_pops is 0
                if sum(valid_tract_pops) == 0:
                    continue

                agg_type = statistics[statistics['agg_var'] == column]['type'].values[0]
                election_data[column] = aggregation.aggregate_statistics(agg_type, valid_values, valid_tract_pops)
            else:
                # if every value is not nan
                if any(~np.isnan(values)):
                    pops = tract_rows['DP05_0001E'].values
                    percents = tract_rows['percent'].values
                    tract_pops = pops*percents
                    # get the values in values that are not NaN as well as the corresponding values in DP05_0001E and the corresponding percents
                    valid_pops = pops[~np.isnan(values)]
                    valid_tract_pops = tract_pops[~np.isnan(values)]

                    props = valid_values / valid_pops
                    weighted_prop = aggregation.weighted_amean(props, valid_tract_pops)

                    # replace the Nan values in values with tract_pop * weighted_prop
                    for i in range(len(values)):
                        if np.isnan(values[i]):
                            values[i] = pops[i] * weighted_prop

                election_data[column] = aggregation.weighted_sum(values, percents)

    aggregated_data.append(election_data)

# save missing_data to csv
missing_data.to_csv('missing_data.csv', index=False)

0 of 356 rows processed
10 of 356 rows processed
20 of 356 rows processed
30 of 356 rows processed
40 of 356 rows processed
50 of 356 rows processed
60 of 356 rows processed
70 of 356 rows processed
80 of 356 rows processed
90 of 356 rows processed
100 of 356 rows processed
110 of 356 rows processed
120 of 356 rows processed
130 of 356 rows processed
140 of 356 rows processed
150 of 356 rows processed
160 of 356 rows processed
170 of 356 rows processed
180 of 356 rows processed
190 of 356 rows processed
200 of 356 rows processed
210 of 356 rows processed
220 of 356 rows processed
230 of 356 rows processed
240 of 356 rows processed
250 of 356 rows processed
260 of 356 rows processed
270 of 356 rows processed
280 of 356 rows processed
290 of 356 rows processed
300 of 356 rows processed
310 of 356 rows processed
320 of 356 rows processed
330 of 356 rows processed
340 of 356 rows processed
350 of 356 rows processed


In [17]:
aggregated_df = pd.DataFrame(aggregated_data)
aggregated_df = pd.concat([elections_with_tracts['filename'], aggregated_df], axis=1)
aggregated_df

,filename,DP05_0026E,DP02_0047E,DP03_0107E,DP03_0109E,DP03_0097E,DP03_0016E,DP02_0099E,DP03_0061E,DP05_0048E,...,DP05_0012E,DP02_0072E,DP02_0138E,DP02_0112E,DP02_0065E,DP02_0055E,DP02_0129E,DP03_0078E,DP02_0069E,DP03_0087E
0,Berkeley_11022010_CityCouncilDistrict1.csv,599.711488,1.394417e-02,220.330966,502.265039,10006.174301,1071.404646,244.463382,510.145336,0.000000,...,766.059773,2068.132691,349.672715,2770.206741,3160.934680,652.642396,406.409091,41.277732,427.358892,115995.068370
1,Berkeley_11042014_CityCouncilDistrict1.csv,799.234210,0.000000e+00,414.399108,447.071733,11916.327712,1272.429326,292.927948,646.849766,0.032167,...,991.122227,2399.559731,387.139750,3417.499289,3597.629633,867.048056,406.910792,59.222937,404.783423,131402.961824
2,Berkeley_11062018_CityCouncilDistrict1.csv,5759.300285,5.295772e-03,905.624193,321.247939,11801.493575,1818.717541,269.229276,1132.676380,300.932283,...,1996.835249,1092.079628,65.771046,14135.455329,3878.417927,122.704295,113.873022,4.414109,12152.574834,179585.690520
3,Minneapolis_11022021_CityCouncilWard1.csv,15395.647974,2.058586e-08,2232.041633,987.502280,24323.691541,2601.227310,350.796561,1428.861369,35.000000,...,3933.759818,3442.266113,67.013676,31921.538465,8192.354186,333.491223,286.232418,184.300498,28676.371205,119437.433084
4,Minneapolis_11052013_Ward13CityCouncil.csv,1182.560568,1.099903e+01,1683.165144,1030.503059,23680.682395,3200.300555,240.723146,253.870559,0.000000,...,1723.523420,6131.024223,2273.123311,6103.760272,3547.622299,2394.932541,1040.340608,654.204113,1615.009777,78113.381587
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
351,Alaska_11082022_SenateDistrictP.csv,10515.518044,5.700873e+00,1606.865133,869.549048,16364.345062,3347.066219,793.607665,768.383348,19.367936,...,2862.707267,3846.046652,8.138817,23850.670632,2734.059570,343.084099,331.643471,149.752077,19036.252703,104965.018489
352,Alaska_11082022_SenateDistrictQ.csv,15028.173273,1.271021e+02,2137.247536,753.833511,26502.758075,5775.171198,979.422774,2048.374269,116.251512,...,3872.778528,4187.110695,81.499022,33664.018991,4263.964472,411.993705,318.409220,198.456312,24649.940430,142961.256924
353,Alaska_11082022_SenateDistrictR.csv,20790.533320,5.419704e+01,2604.743918,1115.826724,31373.752300,6981.185447,1842.946720,2034.287482,89.380519,...,5154.397926,5766.726192,77.362055,45399.761349,6294.042806,613.267365,578.192290,448.890364,32577.048242,116887.806161
354,Alaska_11082022_SenateDistrictS.csv,11447.035315,1.312769e+02,2782.377337,1626.921641,10325.018649,5096.093151,215.900995,527.762951,18.615135,...,3268.962250,3221.040448,14.955394,26587.581242,1404.621508,649.718733,81.772779,366.140580,20209.540922,90522.605214


In [23]:
# open place_acs_data.csv
place_acs_data = pd.read_csv('place_acs_data.csv')

In [26]:
# concat place_acs_data and aggregated_df
elections_data = pd.concat([place_acs_data, aggregated_df], axis=0)
# save as final_data.csv
elections_data.to_csv('final_data.csv', index=False)